# Occasional Updates - Last Update: 2024-05-26

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np

## 2. Update Player Name File

In [5]:
# https://www.tennisabstract.com/mwplayerlist.js?_=1715092352847 // 2024-05-07
names = open('mwplayerlist.txt', encoding='utf-8-sig').read()
# process name list and save to a new text file
names = names[17:-3].split("\", \"")
with open("mwplayerlist_processed.txt", "w", encoding='utf-8-sig') as f:
    for name in names:
        f.write(name)
        f.write('\n')
f.close()

In [6]:
# Separate women and men
mp = []
wp = []
for name in names:
    if name[1] == 'M':
        mp.append(name)
    elif name[1] == 'W':
        wp.append(name)

with open('mp.txt', 'w', encoding='utf-8-sig') as f:
    for name in mp:
        f.write(name)
        f.write('\n')
f.close()

with open('wp.txt', 'w', encoding='utf-8-sig') as f:
    for name in wp:
        f.write(name)
        f.write('\n')
f.close()

## 3. Update Leaderboard Files

In [12]:
# load txt files
def load_txt(file):
    with open(file, 'r', encoding='utf-8-sig') as f:
        attributes = f.readline()[:-1].split(',')
        df = pd.DataFrame([line[:-1].split(',') for line in f], columns = attributes)
    f.close()
    return df

# define functions to clean df
def clean_df(df):
    cleaned = df.replace('', np.NaN)
    cleaned['Date'] = pd.to_datetime(cleaned['Date'])
    return cleaned

In [ ]:
# Update totals (gs + atp1000 + finale + olympic), wr, gs, atp1000 + finale
pl = []
with open('mp.txt') as f:
    for line in f:
        pl.append(line[4:-1].strip())
f.close()

attrs = ['Player', 'Total_Titles', 'Total_Titles_Text', 'GS', 'GS_Text', 'ATP1000', 'ATP Finale', 'Olympics', 'ATP1000+Finale', 'ATP1000+Finale_Text', 'WR', 'W', 'Matches']
dict = {}
for attr in attrs:
    dict[attr] = []

gs = ['Wimbledon', 'US Open', 'Australian Open', 'Roland Garros']
atp1000 = ['Indian Wells Masters', 'Miami Masters', 'Monte Carlo Masters', 'Madrid Masters', 'Rome Masters', 'Canada Masters', 'Cincinnati Masters', 'Shanghai Masters', 'Paris Masters', 'Hamburg Masters']

for i, p in enumerate(pl):
    try:
        df = load_txt('m/matches/txt/'+p+'.txt')
    except:
        print('Finished:', i+1)
        continue
    dict['Player'].append(p)
    dict['Total_Titles'].append(len(df[(df['Tournament'].isin(gs+atp1000+['Tour Finals', 'Masters Cup']) | df['Tournament'].str.contains('Olympics')) & (df['Rd'] == 'F') & (df['W'] == '1')]))
    dict['Total_Titles_Text'].append(str(len(df[(df['Tournament'].isin(gs)) & (df['Rd'] == 'F') & (df['W'] == '1')])) + ' + ' + str(len(df[(df['Tournament'].isin(atp1000)) & (df['Rd'] == 'F') & (df['W'] == '1')])) + ' + ' + str(len(df[(df['Tournament'].isin(['Tour Finals', 'Masters Cup'])) & (df['Rd'] == 'F') & (df['W'] == '1')])) + ' + ' + str(len(df[(df['Tournament'].str.contains('Olympics')) & (df['Rd'] == 'F') & (df['W'] == '1')])))
    dict['GS'].append(len(df[(df['Tournament'].isin(gs)) & (df['Rd'] == 'F') & (df['W'] == '1')]))
    dict['GS_Text'].append(str(len(df[(df['Tournament'] == 'Australian Open') & (df['Rd'] == 'F') & (df['W'] == '1')])) + ' + ' + str(len(df[(df['Tournament'] == 'Wimbledon') & (df['Rd'] == 'F') & (df['W'] == '1')])) + ' + ' + str(len(df[(df['Tournament'] == 'Roland Garros') & (df['Rd'] == 'F') & (df['W'] == '1')])) + ' + ' + str(len(df[(df['Tournament'] == 'US Open') & (df['Rd'] == 'F') & (df['W'] == '1')])))
    dict['ATP1000'].append(len(df[(df['Tournament'].isin(atp1000)) & (df['Rd'] == 'F') & (df['W'] == '1')]))
    dict['Olympics'].append(len(df[(df['Tournament'].str.contains('Olympics')) & (df['Rd'] == 'F') & (df['W'] == '1')]))
    dict['ATP Finale'].append(len(df[(df['Tournament'].isin(['Tour Finals', 'Masters Cup'])) & (df['Rd'] == 'F') & (df['W'] == '1')]))
    dict['ATP1000+Finale'].append(len(df[(df['Tournament'].isin(atp1000+['Tour Finals', 'Masters Cup'])) & (df['Rd'] == 'F') & (df['W'] == '1')]))
    dict['ATP1000+Finale_Text'].append(str(len(df[(df['Tournament'].isin(atp1000)) & (df['Rd'] == 'F') & (df['W'] == '1')])) + ' + ' + str(len(df[(df['Tournament'].isin(['Tour Finals', 'Masters Cup'])) & (df['Rd'] == 'F') & (df['W'] == '1')])))
    try:
        dict['WR'].append(len(df[df['W'] == '1']) / len(df))
    except:
        dict['WR'].append(0)
    dict['W'].append(len(df[df['W'] == '1']))
    dict['Matches'].append(len(df))

    print('Finished:', i+1)

df = pd.DataFrame(dict)

df.to_csv('m/basics/atptitleswr.txt', index = False, sep = ',', encoding = 'utf-8-sig')

In [11]:
# Update the Top10 leaders
df.sort_values(by=['Total_Titles'], ascending=False).reset_index(drop = True).iloc[:10].to_csv('m/leaders/totaltitlestop10.txt', index = False, sep = ',', encoding = 'utf-8-sig')
df[df['Matches'] >= 100].sort_values(by=['WR'], ascending=False).reset_index(drop = True).iloc[:10].to_csv('m/leaders/wrtop10.txt', index = False, sep = ',', encoding = 'utf-8-sig')
df.sort_values(by=['GS'], ascending=False).reset_index(drop = True).iloc[:10].to_csv('m/leaders/gstop10.txt', index = False, sep = ',', encoding = 'utf-8-sig')
df.sort_values(by=['ATP1000+Finale'], ascending=False).reset_index(drop = True).iloc[:10].to_csv('m/leaders/atp1000+finaletop10.txt', index = False, sep = ',', encoding = 'utf-8-sig')

## 4. Update GS Map and Time Trend Chart

In [ ]:
# 1990 - 2024 cumulated GS Titles collection for time trend
import os
files = os.listdir('m/matches/txt/')
attrs = ['Player'] + [i+1990 for i in range(35)]
gs = ['Wimbledon', 'US Open', 'Australian Open', 'Roland Garros']

dict = {}
for attr in attrs:
    dict[attr] = []

for i, file in enumerate(files):
    try:
        df = clean_df(load_txt('m/matches/txt/'+file))
        dict['Player'].append(file.split('.')[0])
        for d in attrs[1:]:
            dict[d].append(len(df[(df['Date'] <= str(d)+'-12-31') & (df['Tournament'].isin(gs)) & (df['Rd'] == 'F') & (df['W'] == '1')]))
        print('Finished:', i+1)
    except:
        pass

df = pd.DataFrame(dict).sort_values(by=2024, ascending=False)
df.to_csv('m/titles/gscounts19902024.txt', index = False, sep = ',', encoding='utf-8-sig')

In [ ]:
# update data for making gs netwrok
import os
files = os.listdir('m/matches/txt/')
attrs = ['Source', 'Target', 'Value']
gs = ['Wimbledon', 'US Open', 'Australian Open', 'Roland Garros']

dict = {}
for attr in attrs:
    dict[attr] = []

for i, file in enumerate(files):
    try:
        df = load_txt('m/matches/txt/'+file)
        target = file.split('.')[0]
        for g in gs:
            cnt = len(df[(df['Tournament'] == g) & (df['Rd'] == 'F') & (df['W'] == '1')])
            if cnt > 0:
                dict['Source'].append(g)
                dict['Target'].append(target)
                dict['Value'].append(cnt)
        print('Finished:', i+1)
    except:
        pass

df = pd.DataFrame(dict)
df.to_csv('m/titles/gsmapping.txt', index = False, sep = ',', encoding='utf-8-sig')

In [15]:
# Get all gs champions
names = [i for i in set(dict['Target'])]

# Save to txt (to paste on Flourish: Mapping - Points)
with open('m/titles/gschampions.txt', 'w') as f:
    for name in names:
        f.write(name)
        f.write('\n')
f.close()

In [16]:
# calculate how many gs, save to txt (to paste on Flourish: Mapping - Points)
cnt = []
with open('m/titles/gscnt.txt', 'w') as f:
    for name in names:
        df = load_txt('m/matches/txt/'+name+'.txt')
        gs = ['Wimbledon', 'US Open', 'Australian Open', 'Roland Garros']
        f.write(str(len(df[(df['Tournament'].isin(gs)) & (df['Rd'] == 'F') & (df['W'] == '1')])))
        f.write('\n')
f.close()

In [21]:
# calculate how many GSs are held
gslist = []
with open('m/titles/gsmapping.txt', 'r') as f:
    f.readline()
    for line in f:
        gslist.append((line.split(',')[0], line.split(',')[2]))

# Print results
for item in ['Wimbledon', 'US Open', 'Australian Open', 'Roland Garros']:
    v = sum([int(gs[1]) for gs in gslist if gs[0] == item])
    print(item, v)

total = sum([int(gs[1]) for gs in gslist])
total

Wimbledon 84
US Open 86
Australian Open 80
Roland Garros 84


334

## 5. Update Data for Simple ML Analysis

In [ ]:
# Set attributes
attrs = ['Player', 'Weight (kg)', 'Height (cm)', 'Birthday', 'Year Turned Pro', 'Right-Left-Handed', 'Two-One-Backhanded', 'Matches', 'Wins', 'Losses', 'Winning Ratio', 'Ace (%)', 'Double Fault (%)', '1st Serve In (%)', '1st Serve Win (%)', '2nd Serve Win (%)', 'Total Points Win (%)', 'Return Points Win (%)', 'Dominant Rate']
dict = {}
for attr in attrs:
    dict[attr] = []

unmatched = []

# Here we want to match the atpprofilebasics with the tennisabstract file 
# by name, so previously we replace - with space in the profile file
df_basics = load_txt('m/basics/atpprofilebasics.txt')
players = [p for p in df_basics['Player']]
for i, player in enumerate(players):
    try:
        df_match = load_txt('m/matches/txt/'+player+'.txt')
    except:
        print(player, 'not found.')
        unmatched.append(player)
        continue
    # add player
    dict['Player'].append(player)
    # add weight
    if df_basics['Weight (kg)'][i] != '':
        dict['Weight (kg)'].append(float(df_basics['Weight (kg)'][i]))
    else:
        dict['Weight (kg)'].append(0)
    # add height
    if df_basics['Height (cm)'][i] != '':
        dict['Height (cm)'].append(float(df_basics['Height (cm)'][i]))
    else:
        dict['Height (cm)'].append(0)
    # add birthday
    dict['Birthday'].append(df_basics['Birthday'][i])
    # add turned pro
    dict['Year Turned Pro'].append(df_basics['Turned Pro'][i])
    # add right-left, 1-0
    if 'Right' in df_basics['Plays'][i]:
        dict['Right-Left-Handed'].append(1)
    elif 'Left' in df_basics['Plays'][i]:
        dict['Right-Left-Handed'].append(0)
    else:
        dict['Right-Left-Handed'].append('')
    # add backhand style
    if 'Two' in df_basics['Plays'][i]:
        dict['Two-One-Backhanded'].append(1)
    elif 'One' in df_basics['Plays'][i]:
        dict['Two-One-Backhanded'].append(0)
    else:
        dict['Two-One-Backhanded'].append('')
    # add matches
    dict['Matches'].append(len(df_match))
    # add wins
    dict['Wins'].append(len(df_match[df_match['W'] == '1']))
    # add losses
    dict['Losses'].append(len(df_match[df_match['W'] == '0']))
    # WR
    try:
        dict['Winning Ratio'].append(len(df_match[df_match['W'] == '1']) / len(df_match))
    except:
        dict['Winning Ratio'].append(0)
    # add ace
    try:
        dict['Ace (%)'].append(sum([float(a[:-1]) for a in df_match['A%'] if '%' in a]) / len(df_match[df_match['A%'].str.contains('%')]))
    except:
        dict['Ace (%)'].append(0)
    # add double fault
    try:
        dict['Double Fault (%)'].append(sum([float(a[:-1]) for a in df_match['DF%'] if '%' in a]) / len(df_match[df_match['DF%'].str.contains('%')]))
    except:
        dict['Double Fault (%)'].append(0)
    # add 1stServeIn
    try:
        dict['1st Serve In (%)'].append(sum([float(a[:-1]) for a in df_match['1stIn'] if '%' in a]) / len(df_match[df_match['1stIn'].str.contains('%')]))
    except:
        dict['1st Serve In (%)'].append(0)
    # add 1stServeWin
    try:
        dict['1st Serve Win (%)'].append(sum([float(a[:-1]) for a in df_match['1st%'] if '%' in a]) / len(df_match[df_match['1st%'].str.contains('%')]))
    except:
        dict['1st Serve Win (%)'].append(0)
    # add 2ndServeWin
    try:
        dict['2nd Serve Win (%)'].append(sum([float(a[:-1]) for a in df_match['2nd%'] if '%' in a]) / len(df_match[df_match['2nd%'].str.contains('%')]))
    except:
        dict['2nd Serve Win (%)'].append(0)
    # add TPW
    try:
        dict['Total Points Win (%)'].append(sum([float(a[:-1]) for a in df_match['TPW'] if '%' in a]) / len(df_match[df_match['TPW'].str.contains('%')]))
    except:
        dict['Total Points Win (%)'].append(0)
    # add RPW
    try:
        dict['Return Points Win (%)'].append(sum([float(a[:-1]) for a in df_match['RPW'] if '%' in a]) / len(df_match[df_match['RPW'].str.contains('%')]))
    except:
        dict['Return Points Win (%)'].append(0)
    # add DR
    try:
        dict['Dominant Rate'].append(sum([float(a) for a in df_match['DR'] if '.' in a]) / len(df_match[df_match['DR'].str.contains('.')]))
    except:
        dict['Dominant Rate'].append(0)
    print('Finished:', i+1)

# save to txt
df = pd.DataFrame(dict)
df.to_csv('m/ds/atpds.txt', index = False, sep = ',', encoding = 'utf-8-sig')
with open('m/basics/unmatched.txt', 'w', encoding = 'utf-8-sig') as f:
    for u in unmatched:
        f.write(u)
        f.write('\n')
f.close()

df_selected = df[(df['Height (cm)'] < 300) & (df['Height (cm)'] > 100) & (df['Weight (kg)'] > 40) & (df['Weight (kg)'] < 150) & (df['Right-Left-Handed'] != '') & (df['Two-One-Backhanded'] != '') & (df['Matches'] >= 100) & (df['Wins'] > 0) & (df['Losses'] > 0) & (df['Winning Ratio'] > 0) & (df['Ace (%)'] > 0) & (df['Double Fault (%)'] > 0) & (df['1st Serve In (%)'] > 0) & (df['1st Serve Win (%)'] > 0) & (df['2nd Serve Win (%)'] > 0) & (df['Total Points Win (%)'] > 0) & (df['Return Points Win (%)'] > 0) & (df['Dominant Rate'] > 0)]
df_selected.to_csv('m/ds/atpdsselected.txt', index = False, sep = ',', encoding = 'utf-8-sig')